In [1]:
import psycopg2, requests, pandas as pd
from config import host, user, password, db_name, api_key
from sqlalchemy import create_engine
from datetime import date
from dateutil.relativedelta import relativedelta
import time
import json
from bs4 import BeautifulSoup
import lxml

In [2]:
def set_connection ():
    connection = psycopg2.connect(database = db_name,
    host = host,
    user = user,
    password = password     
    )
    return connection

  

In [3]:
engine = create_engine(f"postgresql://{user}:{password}@{host}/{db_name}")

Script for getting Nasdaq 100 tickers

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.64 Safari/537.36'}

In [108]:
list_of_endpoints = ["", "&r=21", "&r=41", "&r=61", "&r=81"]
list_of_tickers = []
sleep_period = 0
for i in list_of_endpoints:
    file_to_parse = requests.get(f"https://finviz.com/screener.ashx?v=111&f=exch_nasd,geo_usa,idx_ndx{i}", headers=headers).text
    soup = BeautifulSoup(file_to_parse, "lxml")
    lines_tr_full = soup.find_all("tr", valign = "top")
    for lines_tr in lines_tr_full:
        lines_td = lines_tr.find("td", align = "left").a.text
        list_of_tickers.append(lines_td)
    sleep_period+=2
    time.sleep(sleep_period) 
df = pd.DataFrame(list_of_tickers, columns=["ticker"])
df.to_sql("tickers", engine, if_exists="append", index=False)

93

Get all tickers from database

In [5]:
connection = set_connection() 
try:
    with connection.cursor() as cursor:
        cursor.execute("SELECT ticker FROM tickers")
        alltickers = cursor.fetchall()
except Exception as exc:
    print (f"INFO: Exception occured while woring to database {exc}")
finally:
    if connection:
        connection.close()
        print("[INFO] the connection was closed")
alltickers2 = [i[0] for i in alltickers]
print(alltickers2)

[INFO] the connection was closed
['AAPL', 'ABNB', 'ADBE', 'ADI', 'ADP', 'ADSK', 'AEP', 'ALGN', 'AMAT', 'AMD', 'AMGN', 'AMZN', 'ANSS', 'ATVI', 'AVGO', 'BIIB', 'BKNG', 'BKR', 'CDNS', 'CEG', 'CHTR', 'CMCSA', 'COST', 'CPRT', 'CRWD', 'CSCO', 'CSGP', 'CSX', 'CTAS', 'CTSH', 'DDOG', 'DLTR', 'DXCM', 'EA', 'EBAY', 'ENPH', 'EXC', 'FANG', 'FAST', 'FISV', 'FTNT', 'GFS', 'GILD', 'GOOG', 'GOOGL', 'HON', 'IDXX', 'ILMN', 'INTC', 'INTU', 'ISRG', 'KDP', 'KHC', 'KLAC', 'LCID', 'LRCX', 'MAR', 'MCHP', 'MDLZ', 'META', 'MNST', 'MRNA', 'MRVL', 'MSFT', 'MU', 'NFLX', 'NVDA', 'ODFL', 'ORLY', 'PANW', 'PAYX', 'PCAR', 'PEP', 'PYPL', 'QCOM', 'REGN', 'RIVN', 'ROST', 'SBUX', 'SGEN', 'SIRI', 'SNPS', 'TMUS', 'TSLA', 'TXN', 'VRSK', 'VRTX', 'WBA', 'WBD', 'WDAY', 'XEL', 'ZM', 'ZS']


Script for getting metrics for Nasdaq 100 tickers

In [6]:
list_of_metrics = []
tickers_for_list_of_metrics = []
names_for_list_of_metrics = []
for ticker in alltickers2:
    file_to_parse = requests.get(f"https://finviz.com/quote.ashx?t={ticker}&p=d", headers=headers).text
    soup = BeautifulSoup(file_to_parse, "lxml")
    lines_tr_full = soup.find_all("td", class_ = "snapshot-td2")
    for i in lines_tr_full:
        tickers_for_list_of_metrics.append(ticker)
        list_of_metrics.append(i.find("b").text)
    lines_tr_full = soup.find_all("td", class_ = "snapshot-td2-cp")
    for i in lines_tr_full:
        names_for_list_of_metrics.append(i.text)
    time.sleep(3)
cap_list = []
tickers_for_cap_list = []
for i in range(6, len(list_of_metrics), 72):
    cap_list.append(list_of_metrics[i][:-1])
for i in range(6, len(tickers_for_list_of_metrics),72):
    tickers_for_cap_list.append(tickers_for_list_of_metrics[i])

In [13]:
cap_list = []
tickers_for_cap_list = []
for i in range(6, len(list_of_metrics), 72):
    cap_list.append(list_of_metrics[i][:-1])
for i in range(6, len(tickers_for_list_of_metrics),72):
    tickers_for_cap_list.append(tickers_for_list_of_metrics[i])

In [14]:
print(len(tickers_for_cap_list))
print(len(cap_list))

93
93


In [15]:
df = pd.DataFrame({'ticker':tickers_for_cap_list, 'market_cap':cap_list})
df.to_sql("market_cap", engine, if_exists="append", index=False)

93

In [31]:
print(len(tickers_for_list_of_metrics))
print(len(list_of_metrics))
print(len(names_for_list_of_metrics ))

6696
6696
6696


In [32]:
df = pd.DataFrame({'ticker':tickers_for_list_of_metrics, 'metrics':list_of_metrics, 'name':names_for_list_of_metrics})
df.to_sql("metrics_fin", engine, if_exists="append", index=False)

696